In [1]:
import numpy as np
from gymnasium.envs.mujoco.pusher_v5 import PusherEnv
from stable_baselines3 import PPO
import numpy as np


class CRLPusherEnv(PusherEnv):
    def __init__(self, **kwargs):
        # Inicializamos el entorno original Pusher-v5
        super().__init__(**kwargs)
        
        # Estado interno de la tarea
        self.task_mode = "steel"
        
        # Cacheamos los IDs correctamente
        try:
            # 1. Obtenemos el ID del CUERPO (Body) llamado "object"
            self.obj_body_id = self.model.body("object").id
            
            # 2. Obtenemos el ID de la GEOMETRÍA (Geom) asociada a ese cuerpo
            # body_geomadr nos da la dirección de la primera geometría de ese cuerpo
            self.obj_geom_id = self.model.body_geomadr[self.obj_body_id]
            
            print(f"IDs encontrados -> Body: {self.obj_body_id}, Geom: {self.obj_geom_id}")
            
        except KeyError:
            print("Error Crítico: No se encontró el cuerpo 'object' en el XML.")
            # Es mejor lanzar el error para no seguir si esto falla
            raise

    def set_task(self, mode):
        """
        Cambia dinámicamente la física del entorno para simular no-estacionariedad.
        """
        self.task_mode = mode
        
        if mode == "steel":
            # 1. Modificar Masa (Hacerlo pesado)
            self.model.body_mass[self.obj_body_id] = 5.0 
            
            # 2. Modificar Fricción (Hacerlo rugoso/difícil de mover)
            # geom_friction es un array [sliding, torsional, rolling]. Tocamos sliding.
            self.model.geom_friction[self.obj_geom_id, 0] = 2.0 
            
            print(f"--> Tarea Cambiada: ACERO (Masa: 5.0, Fricción: 2.0)")
            
        elif mode == "glass":
            # 1. Modificar Masa (Hacerlo ligero)
            self.model.body_mass[self.obj_body_id] = 0.1
            
            # 2. Modificar Fricción (Hacerlo resbaladizo como hielo/vidrio)
            self.model.geom_friction[self.obj_geom_id, 0] = 0.1
            
            print(f"--> Tarea Cambiada: CRISTAL (Masa: 0.1, Fricción: 0.1)")

    def reset(self, *, seed=None, options=None):
        obs, info = super().reset(seed=seed, options=options)
        if options is not None and "task_mode" in options:
            self.set_task(options["task_mode"])
        return obs, info
    
    def step(self, action):
        # 1. Ejecutar la física original de Gymnasium
        # Esto calcula reward_dist, reward_ctrl, etc.
        observation, reward, terminated, truncated, info = super().step(action)
        
        # 2. Inyectar nuestra Lógica de Recompensa Dinámica
        if self.task_mode == "glass":
            # Obtenemos la velocidad lineal del objeto desde los datos de simulación
            # cvel devuelve [rotational_vel, linear_vel] (6 valores)
            # Tomamos los últimos 3 (velocidad lineal x, y, z)
            obj_velocity = self.data.cvel[self.obj_body_id][3:]
            vel_magnitude = np.linalg.norm(obj_velocity)
            
            # Penalización por velocidad/impacto
            # Si se mueve muy rápido, castigamos drásticamente.
            # El factor 2.0 es un hiperparámetro que deberás ajustar.
            fragile_penalty = 2.0 * (vel_magnitude ** 2)
            
            # Restamos la penalización a la recompensa original
            #reward -= fragile_penalty
            
            # Agregamos info para monitoreo (útil para TensorBoard)
            info["penalty_fragile"] = fragile_penalty
            
        return observation, reward, terminated, truncated, info

In [ ]:
import os
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.results_plotter import load_results, ts2xy

LOG_DIR = "./logs/pusher_baseline"
os.makedirs(LOG_DIR, exist_ok=True)

# 1) Train env (SIN render)
train_env = gym.make("Pusher-v5")
train_env = Monitor(train_env, LOG_DIR)

# 2) Eval env separado (SIN render)
eval_env = gym.make("Pusher-v5")
eval_env = Monitor(eval_env, os.path.join(LOG_DIR, "eval_monitor"))

model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    tensorboard_log=os.path.join(LOG_DIR, "tb"),  # TensorBoard :contentReference[oaicite:7]{index=7}
)

# Eval antes (baseline)
mean_before, std_before = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"[BEFORE] mean_return={mean_before:.2f} +/- {std_before:.2f}")

TOTAL_STEPS = 1_000_000
model.learn(total_timesteps=TOTAL_STEPS)

# Eval después
mean_after, std_after = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"[AFTER ] mean_return={mean_after:.2f} +/- {std_after:.2f}")

# 3) Curva de aprendizaje (episodic return vs timesteps) desde Monitor logs :contentReference[oaicite:8]{index=8}
x, y = ts2xy(load_results(LOG_DIR), "timesteps")
plt.plot(x, y)
plt.xlabel("timesteps")
plt.ylabel("episode return")
plt.title("PPO on Pusher-v5 (training curve)")
plt.tight_layout()
plt.show()

train_env.close()
eval_env.close()

Using cuda device
Wrapping the env in a DummyVecEnv.


/home/raul/Escritorio/extra/misis/.venv/lib/python3.12/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


[BEFORE] mean_return=-52.44 +/- 3.42
Logging to ./logs/pusher_baseline/tb/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -119     |
| time/              |          |
|    fps             | 1612     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 100        |
|    ep_rew_mean          | -120       |
| time/                   |            |
|    fps                  | 1225       |
|    iterations           | 2          |
|    time_elapsed         | 3          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01165331 |
|    clip_fraction        | 0.109      |
|    clip_range           | 0.2        |
|    entropy_loss         | -9.9       |
|    e

KeyboardInterrupt: 

In [14]:
env = gym.make("Pusher-v5", render_mode="human")
done = False
obs, info = env.reset()
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    #done = terminated or truncated
env.close()

/home/raul/Escritorio/extra/misis/.venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


KeyboardInterrupt: 